In [ ]:
# 导入BeautifulSoup库，用于解析HTML文档
from bs4 import BeautifulSoup

# 打开并读取本地HTML文件 "data\Trending repositories on GitHub today.html"
with open("data\Trending repositories on GitHub today.html") as f:
    html = f.read()

# 使用BeautifulSoup解析HTML内容，并指定解析器为"html.parser"
soup = BeautifulSoup(html, "html.parser")

# 遍历HTML文档中的所有标签
for i in soup.find_all(True):
    # 获取当前标签的所有属性名称
    for name in list(i.attrs):
        # 如果当前属性不是"class"且该属性有值，则删除该属性
        if i[name] and name not in ["class"]:
            del i[name]

# 遍历HTML文档中所有的<svg>, <img>, <video>, <audio>标签
for i in soup.find_all(["svg", "img", "video", "audio"]):
    # 删除这些标签及其内容
    i.decompose()

# 将处理后的HTML内容写入新的文件 "github-trending-slim.html"
with open("github-trending-slim.html", "w") as f:
    f.write(str(soup))

解析html

In [2]:
# 导入必要的库
import aiohttp
import asyncio
from bs4 import BeautifulSoup

def fetch_html(url):
    """
    读取本地HTML文件并返回其内容。
    
    参数:
    url (str): HTML文件的路径。
    
    返回:
    str: 文件中的HTML内容。
    """
    with open(url, encoding="utf-8") as f:
        html = f.read()
    return html

async def parse_github_trending(html):
    """
    解析GitHub趋势页面的HTML内容，提取仓库信息。
    
    参数:
    html (str): GitHub趋势页面的HTML内容。
    
    返回:
    list: 包含多个字典的列表，每个字典包含一个仓库的信息。
    """
    soup = BeautifulSoup(html, 'html.parser')  # 使用BeautifulSoup解析HTML内容

    repositories = []  # 存储所有仓库信息的列表

    # 遍历页面中所有的文章元素（每个文章代表一个仓库）
    for article in soup.select('article.Box-row'):
        repo_info = {}  # 存储单个仓库信息的字典
        
        # 提取仓库名称和URL
        repo_info['name'] = article.select_one('h2 a').text.strip()  # 获取仓库名称并去除多余空白
        repo_info['url'] = article.select_one('h2 a')['href'].strip()  # 获取仓库链接并去除多余空白

        # 提取仓库描述
        description_element = article.select_one('p')
        repo_info['description'] = description_element.text.strip() if description_element else None  # 如果有描述则获取，否则为None

        # 提取编程语言
        language_element = article.select_one('span[itemprop="programmingLanguage"]')
        repo_info['language'] = language_element.text.strip() if language_element else None  # 如果有语言信息则获取，否则为None

        # 提取星星数和Fork数
        stars_element = article.select('a.Link--muted')[0]  # 第一个Link--muted标签是星星数
        forks_element = article.select('a.Link--muted')[1]  # 第二个Link--muted标签是Fork数
        repo_info['stars'] = stars_element.text.strip()  # 获取星星数并去除多余空白
        repo_info['forks'] = forks_element.text.strip()  # 获取Fork数并去除多余空白

        # 提取今日星星数
        today_stars_element = article.select_one('span.d-inline-block.float-sm-right')
        repo_info['today_stars'] = today_stars_element.text.strip() if today_stars_element else None  # 如果有今日星星数则获取，否则为None

        repositories.append(repo_info)  # 将仓库信息添加到列表中

    return repositories  # 返回所有仓库信息的列表

async def main():
    """
    主函数，负责调用其他函数并打印解析后的仓库信息。
    """
    url = 'data\Trending repositories on GitHub today.html'  # HTML文件路径
    html = fetch_html(url)  # 读取HTML文件内容
    repositories = await parse_github_trending(html)  # 解析HTML内容并获取仓库信息

    # 打印每个仓库的详细信息
    for repo in repositories:
        print(f"Name: {repo['name']}")
        print(f"URL: https://github.com{repo['url']}")
        print(f"Description: {repo['description']}")
        print(f"Language: {repo['language']}")
        print(f"Stars: {repo['stars']}")
        print(f"Forks: {repo['forks']}")
        print(f"Today's Stars: {repo['today_stars']}")
        print()

# 运行主函数
await main()

Name: microsoft /

      OmniParser
URL: https://github.comhttps://github.com/microsoft/OmniParser
Description: A simple screen parsing tool towards pure vision based GUI agent
Language: Jupyter Notebook
Stars: 9,480
Forks: 735
Today's Stars: 2,171 stars today

Name: FujiwaraChoki /

      MoneyPrinterV2
URL: https://github.comhttps://github.com/FujiwaraChoki/MoneyPrinterV2
Description: Automate the process of making money online.
Language: Python
Stars: 8,459
Forks: 841
Today's Stars: 464 stars today

Name: unionlabs /

      union
URL: https://github.comhttps://github.com/unionlabs/union
Description: The trust-minimized, zero-knowledge bridging protocol, designed for censorship resistance, extremely high security, and usage in decentralized finance.
Language: Rust
Stars: 26,378
Forks: 1,637
Today's Stars: 1,446 stars today

Name: cordx56 /

      rustowl
URL: https://github.comhttps://github.com/cordx56/rustowl
Description: Visualize Ownership and Lifetimes in Rust
Language: Rust
Sta

# 实现一个订阅智能体


In [9]:
import aiohttp
from bs4 import BeautifulSoup
import os
from typing import Any, AsyncGenerator, Awaitable, Callable, Dict, Optional
from aiocron import crontab
from pydantic import BaseModel, Field
# from pytz import BaseTzInfo
from metagpt.actions.action import Action
from metagpt.logs import logger
from metagpt.roles import Role
from metagpt.schema import Message
from pydantic import BaseModel, Field
from typing import Dict, Callable, Awaitable, AsyncGenerator
import asyncio
from metagpt.roles import Role  # 假设Role类定义在metagpt.roles模块中
from metagpt.schema import Message  # 假设Message类定义在metagpt.schema模块中
import logging


# fix SubscriptionRunner not fully defined
from metagpt.environment import Environment as _  # noqa: F401

In [ ]:

logger = logging.getLogger(__name__)


class SubscriptionRunner(BaseModel):
    """
    一个简单的包装器，用于使用asyncio管理不同角色的订阅任务。
    
    Example:
        >>> import asyncio
        >>> from metagpt.subscription import SubscriptionRunner
        >>> from metagpt.roles import Searcher
        >>> from metagpt.schema import Message
        >>> async def trigger():
        ...     while True:
        ...         yield Message("the latest news about OpenAI")
        ...         await asyncio.sleep(3600 * 24)
        >>> async def callback(msg: Message):
        ...     print(msg.content)
        >>> async def main():
        ...     pb = SubscriptionRunner()
        ...     await pb.subscribe(Searcher(), trigger(), callback)
        ...     await pb.run()
        >>> asyncio.run(main())
    """

    tasks: Dict[Role, asyncio.Task] = Field(default_factory=dict)  # 存储每个角色对应的异步任务
    class Config:
        arbitrary_types_allowed = True  # 允许任意类型的字段


    async def subscribe(
        self,
        role: Role,
        trigger: AsyncGenerator[Message, None],
        callback: Callable[[Message], Awaitable[None]],
    ):
        """
        订阅一个角色到触发器，并设置回调函数以处理角色的响应。

        Args:
            role (Role): 要订阅的角色。
            trigger (AsyncGenerator[Message, None]): 异步生成器，产生要由角色处理的消息。
            callback (Callable[[Message], Awaitable[None]]): 异步回调函数，接收角色的响应。
        """
        loop = asyncio.get_running_loop()  # 获取当前事件循环

        async def _start_role():
            """内部协程，用于启动角色并处理消息流。"""
            async for msg in trigger:
                print("-------------------------------------")
                print(msg)
                resp = await role.run(msg)  # 角色处理消息并返回响应
                await callback(resp)  # 将响应传递给回调函数

        # 创建任务并将任务添加到tasks字典中
        self.tasks[role] = loop.create_task(_start_role(), name=f"Subscription-{role}")

    async def unsubscribe(self, role: Role):
        """
        取消订阅一个角色，并取消其关联的任务。

        Args:
            role (Role): 要取消订阅的角色。
        """
        task = self.tasks.pop(role, None)  # 从tasks字典中移除任务
        if task:
            task.cancel()  # 取消任务

    async def run(self, raise_exception: bool = True):
        """
        运行所有已订阅的任务，并处理它们的完成或异常。

        Args:
            raise_exception (bool, optional): 是否抛出异常。默认为True。
        
        Raises:
            Exception: 如果raise_exception为True且任务抛出异常，则会重新抛出该异常。
        """
        while True:
            for role, task in list(self.tasks.items()):  # 使用list避免字典大小改变时的问题
                if task.done():  # 检查任务是否已完成
                    if task.exception():  # 检查任务是否有异常
                        if raise_exception:
                            raise task.exception()  # 抛出异常
                        logger.error(
                            f"Task {task.get_name()} run error", exc_info=task.exception()
                        )  # 记录错误日志
                    else:
                        logger.warning(
                            f"Task {task.get_name()} has completed. "
                            "If this is unexpected behavior, please check the trigger function."
                        )  # 记录警告日志
                    self.tasks.pop(role)  # 从tasks字典中移除已完成的任务
                    break
            else:
                await asyncio.sleep(1)  # 如果没有任务完成，则等待1秒
                continue
            break  # 如果有任务完成，则退出循环

In [11]:
# 定义了一个用于生成 GitHub Trending 分析报告的模板字符串。
TRENDING_ANALYSIS_PROMPT = """# Requirements
You are a GitHub Trending Analyst, aiming to provide users with insightful and personalized recommendations based on the latest
GitHub Trends. Based on the context, fill in the following missing information, generate engaging and informative titles, 
ensuring users discover repositories aligned with their interests.

# The title about Today's GitHub Trending
## Today's Trends: Uncover the Hottest GitHub Projects Today! Explore the trending programming languages and discover key domains capturing developers' attention. From ** to **, witness the top projects like never before.
## The Trends Categories: Dive into Today's GitHub Trending Domains! Explore featured projects in domains such as ** and **. Get a quick overview of each project, including programming languages, stars, and more.
## Highlights of the List: Spotlight noteworthy projects on GitHub Trending, including new tools, innovative projects, and rapidly gaining popularity, focusing on delivering distinctive and attention-grabbing content for users.
---
# Format Example


# [Title]

## Today's Trends
Today, ** and ** continue to dominate as the most popular programming languages. Key areas of interest include **, ** and **.
The top popular projects are Project1 and Project2.

## The Trends Categories
1. Generative AI
    - [Project1](https://github/xx/project1): [detail of the project, such as star total and today, language, ...]
    - [Project2](https://github/xx/project2): ...
...

## Highlights of the List
1. [Project1](https://github/xx/project1): [provide specific reasons why this project is recommended].
...

---
# Github Trending
{trending}
"""

# 定义了一个名为 CrawlOSSTrending 的类，继承自 Action 类。
# 该类用于爬取 GitHub Trending 页面的数据。
class CrawlOSSTrending(Action):
    # 定义异步方法 run，接收一个可选参数 url，默认为 GitHub Trending 页面的 URL。
    async def run(self, url: str = "https://github.com/trending"):
        # 解析传入的 URL，确保其格式有效。
        parsed_url = urlparse(url)
        if not parsed_url.scheme or not parsed_url.netloc:
            raise ValueError(f"Invalid URL: {url}")

        # 创建一个 aiohttp.ClientSession 对象，用于发起 HTTP 请求。
        async with aiohttp.ClientSession() as client:
            # 使用 ClientSession 发起 GET 请求，获取 GitHub Trending 页面的 HTML 内容。
            async with client.get(url, proxy=None) as response:
                # 如果响应状态码不是 200，则抛出异常。
                response.raise_for_status()
                # 获取响应的文本内容（HTML）。
                html = await response.text()

        # 使用 BeautifulSoup 解析 HTML 内容。
        soup = BeautifulSoup(html, "html.parser")

        # 初始化一个空列表，用于存储解析后的仓库信息。
        repositories = []

        # 遍历 HTML 中所有的 article.Box-row 元素，每个元素代表一个仓库。
        for article in soup.select("article.Box-row"):
            # 初始化一个字典，用于存储当前仓库的信息。
            repo_info = {}

            # 提取仓库名称，并进行必要的清理操作（去除多余空白字符等）。
            repo_info["name"] = (
                article.select_one("h2 a")
                .text.strip()
                .replace("\n", "")
                .replace(" ", "")
            )
            # 构造仓库的完整 URL。
            repo_info["url"] = (
                "https://github.com" + article.select_one("h2 a")["href"].strip()
            )

            # 提取仓库描述，如果不存在则赋值为 None。
            description_element = article.select_one("p")
            repo_info["description"] = (
                description_element.text.strip() if description_element else None
            )

            # 提取编程语言，如果不存在则赋值为 None。
            language_element = article.select_one('span[itemprop="programmingLanguage"]')
            repo_info["language"] = (
                language_element.text.strip() if language_element else None
            )

            # 提取星星数和 Fork 数。
            stars_element = article.select("a.Link--muted")[0]
            forks_element = article.select("a.Link--muted")[1]
            repo_info["stars"] = stars_element.text.strip()
            repo_info["forks"] = forks_element.text.strip()

            # 提取今日新增星星数，如果不存在则赋值为 None。
            today_stars_element = article.select_one("span.d-inline-block.float-sm-right")
            repo_info["today_stars"] = (
                today_stars_element.text.strip() if today_stars_element else None
            )

            # 将当前仓库的信息添加到 repositories 列表中。
            repositories.append(repo_info)

        # 返回所有仓库的信息。
        return repositories

# 定义了一个名为 AnalysisOSSTrending 的类，继承自 Action 类。
# 该类用于分析 GitHub Trending 数据，并生成推荐内容。
class AnalysisOSSTrending(Action):
    # 定义异步方法 run，接收一个参数 trending，表示从 CrawlOSSTrending 获取到的数据。
    async def run(self, trending: Any):
        # 使用 TRENDING_ANALYSIS_PROMPT 模板格式化 trending 数据，并调用 self._aask 方法返回结果。
        # self._aask 是一个假设的方法，用于生成自然语言文本或进一步处理数据。
        return await self._aask(TRENDING_ANALYSIS_PROMPT.format(trending=trending))
# 定义了一个名为 CrawlOSSTrending 的类，继承自 Action 类。
# 该类用于爬取 GitHub Trending 页面的数据。
from urllib.parse import urlparse


In [36]:
# Role实现
# 对于V0.7 以上的版本，需要把老版本的
# self._init_actions 改为self.set_actions
class OssWatcher(Role):
    def __init__(
        self,
        name="Codey",
        profile="OssWatcher",
        goal="Generate an insightful GitHub Trending analysis report.",
        constraints="Only analyze based on the provided GitHub Trending data.",
    ):
        super().__init__(name=name, profile=profile, goal=goal, constraints=constraints)
        self.set_actions([CrawlOSSTrending, AnalysisOSSTrending])
        self._set_react_mode(react_mode="by_order")

    async def _act(self) -> Message:
        logger.info(f"{self._setting}: ready to {self.rc.todo}")
        # By choosing the Action by order under the hood
        # todo will be first SimpleWriteCode() then SimpleRunCode()
        todo = self.rc.todo

        msg = self.get_memories(k=1)[0]  # find the most k recent messages
        result = await todo.run(msg.content)

        msg = Message(content=str(result), role=self.profile, cause_by=type(todo))
        self.rc.memory.add(msg)
        return msg


In [46]:
async def wxpusher_callback(msg: Message):
    print(msg.content)


async def trigger():
    # 这里设置了只触发五次，也可以用while True 永远执行下去
    for i in range(1):
        yield Message("https://github.com/trending")
        await asyncio.sleep(1)
        #  每隔五秒钟执行一次。
        # 也可以设置为每隔3600 * 24 秒执行一次
        
        
# 看一下trigger生成了什么
async for msg in trigger():
    print(msg.content)


https://github.com/trending


In [47]:
# 运行入口，定义了一个异步函数 main()，作为整个程序的启动点。
async def main():
    # 初始化一个空列表 callbacks，用于存储回调函数。
    callbacks = []

    # 如果 callbacks 列表为空，则定义一个简单的打印回调函数 _print 并添加到 callbacks 列表中。
    if not callbacks:
        async def _print(msg: Message):
            # 打印消息内容，msg 是一个 Message 类型的对象，包含消息的内容和其他元数据。
            print(msg.content)
        # 将 _print 回调函数添加到 callbacks 列表中。
        callbacks.append(_print)

    # 定义一个名为 callback 的异步函数，用于处理来自订阅任务的消息。
    # 这个函数会将所有注册的回调函数应用到传入的消息 msg 上。
    async def callback(msg):
        # 使用 asyncio.gather 并发执行所有回调函数，确保每个回调函数都能接收到相同的消息。
        await asyncio.gather(*(call(msg) for call in callbacks))

    # 创建一个 SubscriptionRunner 实例 runner，用于管理订阅任务。
    runner = SubscriptionRunner()

    # 调用 runner.subscribe 方法，订阅 OssWatcher 角色到触发器 trigger()，并设置回调函数 callback。
    # OssWatcher 是一个角色类，负责爬取和分析 GitHub Trending 数据。
    # trigger() 是一个异步生成器，用于定期生成消息（例如每隔5秒生成一次）。
    # callback 是一个异步函数，用于处理从 OssWatcher 角色接收到的消息。
    await runner.subscribe(OssWatcher(), trigger(), callback)

    # 启动所有已订阅的任务，并持续运行，直到有任务完成或抛出异常。
    # run() 方法会不断检查任务状态，处理已完成的任务或捕获异常。
    await runner.run()

In [48]:

await main()

# [Title]

## Today's Trends
Today, the GitHub Trending list showcases a blend of innovative tools and cutting-edge projects, with ** and ** leading the charge in programming languages. Key areas of interest span from ** to **, highlighting the latest in ** and **.
The top trending projects are ** and **.

## The Trends Categories
1. Generative AI
    - [microsoft/OmniParser](https://github.com/microsoft/OmniParser): A simple screen parsing tool towards pure vision based GUI agent, written in **Jupyter Notebook**. It boasts **11,634 stars** and has seen a significant increase of **2,168 stars today**.
    - [harry0703/MoneyPrinterTurbo](https://github.com/harry0703/MoneyPrinterTurbo): Generate short videos with one click using AI LLM, coded in **Python**. This project has **22,271 stars** and gained **216 stars today**.

2. CLI Tools
    - [spf13/cobra](https://github.com/spf13/cobra): A Commander for modern Go CLI interactions, with **39,230 stars** and a modest gain of **9 stars toda

2025-02-18 22:59:00.856 | WARNING  | metagpt.utils.cost_manager:update_cost:49 - Model glm-4-flash not found in TOKEN_COSTS.


 space.
-------------------------------------
user: https://github.com/trending
# [Title]

## Today's Trends
Today, the GitHub Trending list showcases a blend of innovative tools and cutting-edge projects, with ** and ** leading the charge in programming languages. Key areas of interest span from ** to **, highlighting the latest in ** and **.
The top trending projects are ** and **.

## The Trends Categories
1. Generative AI
    - [microsoft/OmniParser](https://github.com/microsoft/OmniParser): A simple screen parsing tool towards pure vision based GUI agent, written in **Jupyter Notebook**. It boasts **11,634 stars** and has seen a significant increase of **2,168 stars today**.
    - [harry0703/MoneyPrinterTurbo](https://github.com/harry0703/MoneyPrinterTurbo): Generate short videos with one click using AI LLM, coded in **Python**. This project has **22,271 stars** and gained **216 stars today**.

2. CLI Tools
    - [spf13/cobra](https://github.com/spf13/cobra): A Commander for moder

Task Subscription-private_context=None private_config=None private_llm=<metagpt.provider.zhipuai_api.ZhiPuAILLM object at 0x000001F82CFE25C0> name='Codey' profile='OssWatcher' goal='Generate an insightful GitHub Trending analysis report.' constraints='Only analyze based on the provided GitHub Trending data.' desc='' is_human=False role_id='' states=["0. <class '__main__.CrawlOSSTrending'>", "1. <class '__main__.AnalysisOSSTrending'>"] actions=[CrawlOSSTrending, AnalysisOSSTrending] rc=RoleContext(env=None, msg_buffer=MessageQueue(), memory=Memory(storage=[], index=defaultdict(<class 'list'>, {}), ignore_id=False), working_memory=Memory(storage=[], index=defaultdict(<class 'list'>, {}), ignore_id=False), state=-1, todo=None, watch={'metagpt.actions.add_requirement.UserRequirement'}, news=[], react_mode='by_order', max_react_loop=1) addresses={'__main__.OssWatcher', 'Codey'} planner=Planner(plan=Plan(goal='', context='', tasks=[], task_map={}, current_task_id=''), working_memory=Memory(s